In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 38283148
paper_name = 'sulaiman_ramotar_2024' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/table1.txt', header=None, names=['orfs','genes','sensitivity'], sep='\t')

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 12 x 3


In [10]:
original_data.head()

,orfs,genes,sensitivity
0,YMR048W,csm3Δ,+++++
1,YOR198C,bfr1Δ,+++++
2,YLR192C,hcr1Δ,++++++
3,YDL101C,dun1Δ,+++++++
4,YGR167W,clc1Δ,++


In [11]:
original_data['orf'] = original_data['orfs'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orfs, genes, sensitivity, orf]
Index: []


In [17]:
original_data['data'] = original_data['sensitivity'].apply(lambda x: -len(x))

In [19]:
original_data.set_index('orf', inplace=True)

In [20]:
original_data = original_data[['data']].copy()

In [21]:
original_data = original_data.groupby(original_data.index).mean()

In [22]:
original_data.shape

(12, 1)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [22266]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,22266
data_type,value
orf,
YCL061C,-6
YDL101C,-7
YGR167W,-2
YLR192C,-6
YLR358C,-5


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22266
,data_type,value
gene_id,orf,
546,YCL061C,-6
1979,YDL101C,-7
3112,YGR167W,-2
3881,YLR192C,-6
4049,YLR358C,-5


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22266           
data_type       value     valuez
gene_id orf                     
546     YCL061C    -6 -21.427227
1979    YDL101C    -7 -24.998431
3112    YGR167W    -2  -7.142409
3881    YLR192C    -6 -21.427227
4049    YLR358C    -5 -17.856022

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')